<h2 align='center'>理财公告关键信息智能抽取任务解决思路 </h2>
<h3> 1 任务分析 </h3>
<p>初步分析竞赛所给数据可以将本次竞赛任务分为三个子任务：</p>
<b>
    <ol>
        <li>表格信息抽取</li>
        <li>文本信息抽取</li>
        <li>类型字段预测</li>
    </ol>
</b>

### 2 数据分析
<b>通过对数据进行探索性分析，确定需要进行数据预处理的问题以及解决方案</b>
- pdf转文本
  - 文本跨行导致模式识别逻辑无法触发，<b>可以考虑将pdf转html或xml，能够解决一部分上述问题。</b>
  - 文本行合并导致信息无法匹配，<b>探索中</b>
  
- 优先级确定
  - 表格文本记录同时存在，以本次购买触发优先。[11199.pdf](./test_data/11199.PDF)
  - 存在多表格记录，结合表头信息优先本次购买的信息。[11334.pdf](./test_data/11334.PDF)
  - 存在多文本记录，仅抽取购买部分。[12107.pdf](./test_data/12107.PDF)
  
- 理财产品与购买描述信息匹配
  - 多产品多购买描述，数量相同，一一对应即可。
  - 多产品多购买描述，数量不同，探索中。
  - 多产品单描述，全部采用同一条。

<b>此部分最终将数据分为两部分，一部分做文本信息抽取，一部分做表格信息抽取。<br>
    同时，为每条理财产品匹配其购买描述信息，以作为后续关系预测使用</b>

In [1]:
from conf import read_pdf, parse_xml, buy_det_pat, ner

文本触发词判定样例[12210.pdf](./test_data/12210.PDF)

In [2]:
path = "test_data/12210.PDF"
xml_path = "test_data_xml/12210.xml"

content, _ = read_pdf(path)
xml_content = parse_xml(xml_path)

In [3]:
print("pdfplumber 转换结果".center(50, '='))
pdf_match_lines = list()
for line_id, line in enumerate(content):
    if buy_det_pat.search(line):
        pdf_match_lines.append(line)
    print(line_id, line, sep=" : ")
         

=================pdfplumber 转换结果==================
0 : 证券代码：002893     证券简称：华通热力    公告编号：2018-096号 
1 :  
2 : 北京华远意通热力科技股份有限公司 
3 : 关于使用闲置自有资金购买理财产品的进展公告 
4 : 本公司及董事会全体成员保证信息披露内容的真实、准确和完整，没有虚
5 : 假记载、误导性陈述或重大遗漏。 
6 :  
7 : 北京华远意通热力科技股份有限公司（以下简称“公司”）于 2018 年 8 月
8 : 20 日召开了第二届董事会第五次会议，审议通过了《关于调整闲置自有资金现
9 : 金管理投资品种的议案》，同意在确保公司日常运营和资金安全的基础上，由公
10 : 司在风险可控的前提下利用闲置自有资金购买低风险、流动性高的金融机构理财
11 : 产品或结构性存款。上述议案经 2018 年第一次临时股东大会审议通过。详细内
12 : 容参见2018年8月21日巨潮资讯网刊登的《关于调整闲置自有资金现金管理投
13 : 资品种的公告》（公告编号：2018-064号）。 
14 : 一、公司购买理财产品的情况 
15 : 1、2018 年 8 月 14 日，公司与中国光大银行北京东高地支行签订《中国光
16 : 大银行资产管理类理财产品协议书》，以人民币 1,000 万元闲置自有资金购买中
17 : 国光大银行东高地支行机构理财季季盈 EB4324，具体事宜如下： 
18 : （1） 产品名称：机构理财季季盈EB4324 
19 : （2） 产品类型： 非保本浮动收益型产品 
20 : （3）预期最高年化收益率：理财投资资产在正常处置情况下，在扣除理财
21 : 资金托管费、各类费用后，本理财产品的预期年化收益率为 4.60%。 
22 : （3） 起息日：2018年8 月14日 
23 : （4） 到期日：2018年11 月14日 
24 : （5） 理财金额：1,000万元 
25 : （6） 资金来源：闲置自有资金 
26 : （7） 关联关系说明：公司与中国光大银行无关联关系 
27 : 2、2018年8月21日，公司与中国光大银行北京东高地支行签订《中国光大
28 : 银行资产管理类理财产品协议书》，以人民币 1,000 万元闲置自有资金购

In [4]:
print("pdfplumber matched lines count:", len(pdf_match_lines))

pdfplumber matched lines count: 0


In [5]:
print("xml 转换结果".center(50, '='))
xml_match_lines = list()
for line_id, line in enumerate(xml_content):
    if buy_det_pat.search(line):
        xml_match_lines.append(line)
    print(line_id, line, sep=" : ")

=====================xml 转换结果=====================
0 : 证券代码：002893证券简称：华通热力公告编号：2018-096号
1 : 北京华远意通热力科技股份有限公司
2 : 关于使用闲置自有资金购买理财产品的进展公告
3 : 本公司及董事会全体成员保证信息披露内容的真实、准确和完整，没有虚假记载、误导性陈述或重大遗漏。
4 : 北京华远意通热力科技股份有限公司（以下简称“公司”）于2018年8月20日召开了第二届董事会第五次会议，审议通过了《关于调整闲置自有资金现金管理投资品种的议案》，同意在确保公司日常运营和资金安全的基础上，由公司在风险可控的前提下利用闲置自有资金购买低风险、流动性高的金融机构理财产品或结构性存款。上述议案经2018年第一次临时股东大会审议通过。详细内容参见2018年8月21日巨潮资讯网刊登的《关于调整闲置自有资金现金管理投资品种的公告》（公告编号：2018-064号）。
5 : 一、公司购买理财产品的情况
6 : 1、2018年8月14日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
7 : （1）产品名称：机构理财季季盈EB4324
8 : （2）产品类型：非保本浮动收益型产品
9 : （3）预期最高年化收益率：理财投资资产在正常处置情况下，在扣除理财资金托管费、各类费用后，本理财产品的预期年化收益率为4.60%。
10 : （3）起息日：2018年8月14日
11 : （4）到期日：2018年11月14日
12 : （5）理财金额：1,000万元
13 : （6）资金来源：闲置自有资金
14 : （7）关联关系说明：公司与中国光大银行无关联关系
15 : 2、2018年8月21日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
16 : （1）产品名称：机构理财季季盈EB4324
17 : （2）产品类型：非保本浮动收益型产品
18 : （3）预期最高年化收益率：理财投资资产在正常处置情况下，在扣除理财资金托管费、各类费

In [6]:
print("xml matched lines count:", len(xml_match_lines))
for line in xml_match_lines:
    print(line)

xml matched lines count: 9
1、2018年8月14日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
2、2018年8月21日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
3、2018年8月29日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
4、2018年8月30日，公司全资子公司北京华通兴远供热节能技术有限公司（以下简称“华通兴远”）与杭州银行中关村支行签订《卓越增盈第180379期预约39天型（新客专属）》理财协议，以人民币20万元闲置自有资金购买杭州银行中关村支行卓越增盈第180379期预约39天型（新客专属）理财产品，具体事宜如下：
5、2018年9月04日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
6、2018年9月12日，公司与中国建设银行北京丰科园支行签订《中国建设银行股份有限公司理财产品客户协议书》，以人民币1,000万元闲置自有资金购买中国建设银行丰科园支行北京“乾元一升益鑫”2018年207理财产品，具体事宜如下：
7、2018年9月20日，公司与中国建设银行北京丰科园支行签订《中国建设银行股份有限公司理财产品客户协议书》，以人民币20万元闲置自有资金购买中国建设银行丰科园支行北京“乾元一日鑫月溢”开放式资产组合型人民币理财，具体事宜如下：
8、2018年9月21日，公司与兴业银行西城支行签订《兴业银行金雪球添利快线净值型理财产品协议书》，以人民币300万元闲置自有资金购买兴业银行西城支行金雪球添利快线净值型理财产品，具体事宜如下：
9、2018年9月25日，公司与杭州银行中关村支行签订《杭州银行“添利宝”结构性存款产品》，以人民币3,000万元闲置自有资金购买杭州银行中关村支行“

### 3 表格信息抽取
- 确定表格类型
    - 横向表格与纵向表格，对比两者差异即可设计不同提取逻辑
- 表头识别
    - 表头关键字提取，模式匹配
- 单元格跨行
    - 真实跨行，进行填充
    - 转化错误导致换行，进行合并

这部分的难点主要在于表头定位以及单元格跨行处理，解决方法，挑选10个以上各类型样本，分析所转换文本的共同之处，编写处理逻辑。

<p>
    训练数据 1.PDF ->
    <a href="./htmls/1.html" title="1.PDF"> 1.html</a>
    可以使用bs4依据元素进行抽取。
</p>

### 4 文本信息抽取
- 确定抽取起止位置
    - 起始位置，购买类触发词命中即开始。
    - 终止位置，特此公告结束。
- 字段提取
    - Item类型，模式匹配。
    - Sentence类型，模式匹配与命名实体识别。
- 关系抽取
    - 购买方，命名实体识别。
    - 销售方，命名实体识别。
    
**对于命名实体识别，在此提供有两种解决思路：**

    - 使用Python现成工具，如HanLP、StanfordCoreNLP等，
    - 也可以自己训练模型，当前主流的框架为BiLSTM+CRF与BERT+CRF。


In [7]:
for line in xml_content:
    if buy_det_pat.search(line):
        answer = ner(line)
        print(line, answer, sep='\n')
        

1、2018年8月14日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
[[['公司', 'NT'], ['中国光大银行北京东高地支行', 'NT'], ['中国光大银行', 'NT'], ['中国光大银行东高地支行', 'NT']]]
2、2018年8月21日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
[[['公司', 'NT'], ['中国光大银行北京东高地支行', 'NT'], ['中国光大银行', 'NT'], ['中国光大银行东高地支行', 'NT']]]
3、2018年8月29日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
[[['公司', 'NT'], ['中国光大银行北京东高地支行', 'NT'], ['中国光大银行', 'NT'], ['中国光大银行东高地支行', 'NT']]]
4、2018年8月30日，公司全资子公司北京华通兴远供热节能技术有限公司（以下简称“华通兴远”）与杭州银行中关村支行签订《卓越增盈第180379期预约39天型（新客专属）》理财协议，以人民币20万元闲置自有资金购买杭州银行中关村支行卓越增盈第180379期预约39天型（新客专属）理财产品，具体事宜如下：
[[['公司', 'NT'], ['北京华通兴远供热节能技术有限公司', 'NT'], ['华通兴远', 'NT'], ['杭州银行中关村支行', 'NT'], ['杭州银行中关村支行', 'NT'], ['卓越增盈', 'NT']]]
5、2018年9月04日，公司与中国光大银行北京东高地支行签订《中国光大银行资产管理类理财产品协议书》，以人民币1,000万元闲置自有资金购买中国光大银行东高地支行机构理财季季盈EB4324，具体事宜如下：
[[['公司', 'NT'], ['中国光大银行北京东高地支行', 'NT'], ['中国光大银行', 'NT'], ['中国

### 5 字段格式化
- 金额格式化
    - 转换为万元即可，涉及单位处理、中文数字转换等。
- 日期格式化
    - 转换为----/--/--，无需零补位。
    
可能其他字段在抽取时，会包含一些结束符，需要处理。


### 6 类型字段预测
- 实际购买公司与上市公司关系
    - 按单产品，依据其购买信息，使用Baseline模型进行预测。
- 理财产品类型
    - 和Baseline做法一致。

### 7 误差分析
- 随机挑选100条与训练结果不一致的样本，分析归纳原因。
- 重复该步骤。